## Coding Temple Capstone II - Job Openings and Labor Turnover Survey (BLS)
### Harry Hill - May 2023
## Step 1: Data Collection
Dataset: Jop Openings and Labor Turnover Survey, May 2023, US Bureau of Labor Statistics https://www.bls.gov/jlt/data.htm
- myAPI: 2b4ca11bf83746deaa5718b92a582c25

Context: The Bureau of Labor Statistics updates a dataset monthly with employment and job vacancy statistics subdivided by industry, business size, and region/state. https://www.bls.gov/news.release/archives/jolts_05032022.pdf
- Reported survey of a sample of 20,700 businesses across the United States, from a population of over 9.4 million.
- Data is adjusted for regular seasonal changes in these observations.
- Labor statistics are ratio-adjusted using employment statistics from the Current Employment Statistics (CES) survey

Format: Bureau of Labor Statistics Public Data API, public-use; https://www.bls.gov/developers/api_signature_v2.htm#multiple

Defining Question: What are the current trends in the US labor market? 
- Does 'no one want to work' as some claim, or are there willing workers who are not finding employment? 
- Are any US industries experiencing more extreme conditions than others? 
- Are workers voluntarily quitting or are they being fired?
- Is the 'Great Resignation' of 2021 still being felt? (3/2022 record 3% quits rate, 7.1% openings rate)
- Which industries have had major layoffs? https://mondo.com/insights/mass-layoffs-in-2022-whats-next-for-employees/#:~:text=Candy%20Digital%20layoffs%3A%2033%25%20of,laid%20off%20(November%2C%202022)

Hypothesis: If there are openings that are not being taken advantage of in an industry, 
- Job openings & layoffs rate ~ labor demand (full-time, part-time, and temporary positions)
- Quits rate - employment demand

In [60]:
# take list of series IDs, split, put into quotes
line = "JTS000000000000000HIR JTS000000000000000JOR JTS000000000000000LDR JTS000000000000000QUR JTS000000000000000TSR JTS000000000000000UOR JTS100000000000000HIR JTS100000000000000JOR JTS100000000000000LDR JTS100000000000000QUR JTS100000000000000TSR JTS230000000000000HIR JTS230000000000000JOR JTS230000000000000LDR JTS230000000000000QUR JTS230000000000000TSR JTS300000000000000HIR JTS300000000000000JOR JTS300000000000000LDR JTS300000000000000QUR JTS300000000000000TSR JTS400000000000000HIR JTS400000000000000JOR JTS400000000000000LDR JTS400000000000000QUR JTS400000000000000TSR JTS510000000000000HIR JTS510000000000000JOR JTS510000000000000LDR JTS510000000000000QUR JTS510000000000000TSR JTS510099000000000HIR JTS510099000000000JOR JTS510099000000000LDR JTS510099000000000QUR JTS510099000000000TSR JTS540099000000000HIR JTS540099000000000JOR JTS540099000000000LDR JTS540099000000000QUR JTS540099000000000TSR JTS600000000000000HIR JTS600000000000000JOR JTS600000000000000LDR JTS600000000000000QUR JTS600000000000000TSR JTS900000000000000HIR JTS900000000000000JOR JTS900000000000000LDR JTS900000000000000QUR JTS900000000000000TSR"

# 1. transform series IDs with quotes
import re
re.sub(r'(\w+)', r'"\1"', line)

# transform spaced series IDs into neat list with commas and quotes
line2 = ', '.join('"{}"'.format(word) for word in line.split(' '))

print(line2)

"JTS000000000000000HIR", "JTS000000000000000JOR", "JTS000000000000000LDR", "JTS000000000000000QUR", "JTS000000000000000TSR", "JTS000000000000000UOR", "JTS100000000000000HIR", "JTS100000000000000JOR", "JTS100000000000000LDR", "JTS100000000000000QUR", "JTS100000000000000TSR", "JTS230000000000000HIR", "JTS230000000000000JOR", "JTS230000000000000LDR", "JTS230000000000000QUR", "JTS230000000000000TSR", "JTS300000000000000HIR", "JTS300000000000000JOR", "JTS300000000000000LDR", "JTS300000000000000QUR", "JTS300000000000000TSR", "JTS400000000000000HIR", "JTS400000000000000JOR", "JTS400000000000000LDR", "JTS400000000000000QUR", "JTS400000000000000TSR", "JTS510000000000000HIR", "JTS510000000000000JOR", "JTS510000000000000LDR", "JTS510000000000000QUR", "JTS510000000000000TSR", "JTS510099000000000HIR", "JTS510099000000000JOR", "JTS510099000000000LDR", "JTS510099000000000QUR", "JTS510099000000000TSR", "JTS540099000000000HIR", "JTS540099000000000JOR", "JTS540099000000000LDR", "JTS540099000000000QUR", 

In [123]:
import pandas as pd
import requests

# List of desired series from BLS JOLTS 
series_list = ["JTS000000000000000HIR", "JTS000000000000000JOR", "JTS000000000000000LDR", "JTS000000000000000QUR", "JTS000000000000000TSR", "JTS000000000000000UOR", "JTS100000000000000HIR", "JTS100000000000000JOR", "JTS100000000000000LDR", "JTS100000000000000QUR", "JTS100000000000000TSR", "JTS230000000000000HIR", "JTS230000000000000JOR", "JTS230000000000000LDR", "JTS230000000000000QUR", "JTS230000000000000TSR", "JTS300000000000000HIR", "JTS300000000000000JOR", "JTS300000000000000LDR", "JTS300000000000000QUR", "JTS300000000000000TSR", "JTS400000000000000HIR", "JTS400000000000000JOR", "JTS400000000000000LDR", "JTS400000000000000QUR", "JTS400000000000000TSR", "JTS510000000000000HIR", "JTS510000000000000JOR", "JTS510000000000000LDR", "JTS510000000000000QUR", "JTS510000000000000TSR", "JTS510099000000000HIR", "JTS510099000000000JOR", "JTS510099000000000LDR", "JTS510099000000000QUR", "JTS510099000000000TSR", "JTS540099000000000HIR", "JTS540099000000000JOR", "JTS540099000000000LDR", "JTS540099000000000QUR", "JTS540099000000000TSR", "JTS600000000000000HIR", "JTS600000000000000JOR", "JTS600000000000000LDR", "JTS600000000000000QUR", "JTS600000000000000TSR", "JTS900000000000000HIR", "JTS900000000000000JOR", "JTS900000000000000LDR", "JTS900000000000000QUR", "JTS900000000000000TSR"]

# Create an empty DataFrame with columns for series IDs
df = pd.DataFrame(columns=['SeriesID'])

# Specify the start and end year for data retrieval (SPECIFY MONTH TOO?)
start_year = 2013
end_year = 2023

# Iterate over each series ID
for series in series_list:
    # Make a POST request to the BLS API
    data = {
        'seriesid': [series],
        'startyear': str(start_year),
        'endyear': str(end_year),
        'registrationkey': '2b4ca11bf83746deaa5718b92a582c25',
    }
    response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', json=data)
    
    # Extract the data from the response
    if response.status_code == 200:
        result = response.json()
        series_data = result['Results']['series'][0]
        
        # Extract the series title
        series_title = series_data['seriesID']
        
        # Extract the months data
        months_data = series_data['data']
        
        # Create a dictionary to store the row data
        row_data = {'SeriesID': series_title}
        
        # Iterate over each month's data
        for month in months_data:
            year = month['year']
            period = month['period']
            value = month['value']
            
            # Create a column name based on year and period
            column_name = f'{year}-{period}'
            
            # Add the value to the row data
            row_data[column_name] = value
        
        # Add the row data to the DataFrame
        df = df.append(row_data, ignore_index=True)

# Print the DataFrame
print(df)

/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

                 SeriesID 2013-M01 2013-M02 2013-M03 2013-M04 2013-M05  \
0   JTS000000000000000HIR      3.3      3.4      3.2      3.4      3.4   
1   JTS000000000000000JOR      2.8      2.9      2.9      2.9      3.0   
2   JTS000000000000000LDR      1.2      1.2      1.3      1.3      1.3   
3   JTS000000000000000QUR      1.7      1.7      1.6      1.7      1.6   
4   JTS000000000000000TSR      3.2      3.2      3.1      3.3      3.3   
5   JTS000000000000000UOR      3.2      3.0      2.9      2.9      2.8   
6   JTS100000000000000HIR      3.7      3.8      3.6      3.8      3.8   
7   JTS100000000000000JOR      3.0      3.1      3.1      3.0      3.2   
8   JTS100000000000000LDR      1.4      1.4      1.5      1.5      1.5   
9   JTS100000000000000QUR      1.9      1.9      1.8      1.9      1.8   
10  JTS100000000000000TSR      3.6      3.5      3.5      3.6      3.6   
11  JTS230000000000000HIR      5.9      6.5      6.1      5.1      5.8   
12  JTS230000000000000JOR      2.1    

/var/folders/l6/k0bks1dx4wd409pkxgr263rm0000gn/T/ipykernel_55992/4085074310.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


## Step 2: Data Cleaning

### 1. Data Setup

Observations: monthly updated series for specific industry/sector and variable (hires/openings/layoffs/separations)

#### Variable Descriptions: https://www.bls.gov/help/one_screen/JT.htm#select-select-one-or-more-size-classes
1. **Hires** (HIR): "are all additions to the payroll during the month."
2. **Job Openings** (JOR): "are all positions that are open (not filled) on the last business day of the month. The positions can be full-time or part-time, and they can be permanent, short-term, or seasonal. A job is "open" only if it meets all three of the following conditions: 1) a specific position exists and there is work available for that position; 2) the job could start within 30 days, whether or not the establishment finds a suitable candidate during that time; and 3) there is active recruiting for workers from outside the establishment location that has the opening."
3. **Layoffs (and Discharges)** (LDR): "Involuntary separations initiated by the employer."
4. **Quits** (QUR): "Employees who left voluntarily. Exception: retirements or transfers to other locations are reported with Other Separations."
5. **Total Separations** (TSR): "are all employees separated from the payroll during the calendar month."
    - Included in this total but not in the sample I chose is also **'other separations'** which is described as "retirements; transfers to other locations; deaths; or separations due to employee disability."
6. **Unemployed Persons per Job Opening Ratio** (UOR): only for 'total nonfarm' which is the aggregated dataset without subdivisions by industry

This data *is* **seasonally adjusted** which is described as "a statistical technique that attempts to measure and remove the influences of predictable seasonal patterns from time series data. Seasonally adjusted data contain this type of adjustment to enable comparisons of data across any point in time. Not seasonally adjusted data have no such adjustment."

Units of Measurement: **Rate** - "The job openings rate is computed by dividing the number of job openings by the sum of employment and job openings and multiplying that quotient by 100. All other data element rates (hires, total separations, quits, layoffs and discharges, and other separations) are computed by dividing the data element level by employment and multiplying that quotient by 100."

#### Industry/Sector Descriptions: https://www.bls.gov/iag/home.htm

1. Private sectors
    1. Mining/Logging
    2. Construction
    3. Manufacturing
        - (1) durable goods 
        - (2) nondurable goods subsector
    4. Trade, transportation, and utilities
        - (1) wholesale trade
        - (2) retail trade
        - (3) transportation, warehousing, and utilities
    5. Information
        - (1) pulblishing, motion pictures, recording
        - (2) broadcasting, telecommunications
        - (3) data processing
            - programmers, support specialists, systems analysts developers
        - (4) other info services
            - editors, librarians, admin, internet publishing
    6. Financial activties 
        - (1) finance and insurance
        - (2) real estate, rental, and leasing
    7. Professional and business services
        - (1) professional, scientific, and technical services
            * legal, accounting, architectural, design, advertising
        - (2) management of companies and enterprises
            * accounting, auditors, financial managers, supervisors, general managers
        - (3) administrative and support and waste management and remediation services
            * janitors, laborers, movers, landscaping, clerks guards
    8. Education and Health Services
        - (1) educational services
        - (2) health care and social assistance
    9. Leisure and hospitality
        - (1) arts, entertainment, and recreation
        - (2) accomodation and food services
    10. Other services
2. Public / Government
    1. Federal
    2. State and local
        - (1) state and local government education
        - (2) state and local government, excluding education

In [135]:
# transform list of seriesIDs into string of newline separated words
    # for easy next step
list2 = ["JTS000000000000000HIR", "JTS000000000000000JOR", "JTS000000000000000LDR", "JTS000000000000000QUR", "JTS000000000000000TSR", "JTS000000000000000UOR", "JTS100000000000000HIR", "JTS100000000000000JOR", "JTS100000000000000LDR", "JTS100000000000000QUR", "JTS100000000000000TSR", "JTS230000000000000HIR", "JTS230000000000000JOR", "JTS230000000000000LDR", "JTS230000000000000QUR", "JTS230000000000000TSR", "JTS300000000000000HIR", "JTS300000000000000JOR", "JTS300000000000000LDR", "JTS300000000000000QUR", "JTS300000000000000TSR", "JTS400000000000000HIR", "JTS400000000000000JOR", "JTS400000000000000LDR", "JTS400000000000000QUR", "JTS400000000000000TSR", "JTS510000000000000HIR", "JTS510000000000000JOR", "JTS510000000000000LDR", "JTS510000000000000QUR", "JTS510000000000000TSR", "JTS510099000000000HIR", "JTS510099000000000JOR", "JTS510099000000000LDR", "JTS510099000000000QUR", "JTS510099000000000TSR", "JTS540099000000000HIR", "JTS540099000000000JOR", "JTS540099000000000LDR", "JTS540099000000000QUR", "JTS540099000000000TSR", "JTS600000000000000HIR", "JTS600000000000000JOR", "JTS600000000000000LDR", "JTS600000000000000QUR", "JTS600000000000000TSR", "JTS900000000000000HIR", "JTS900000000000000JOR", "JTS900000000000000LDR", "JTS900000000000000QUR", "JTS900000000000000TSR"]
list3 = '\n\t'.join(list2)
print(f"\t{list3}")

	JTS000000000000000HIR
	JTS000000000000000JOR
	JTS000000000000000LDR
	JTS000000000000000QUR
	JTS000000000000000TSR
	JTS000000000000000UOR
	JTS100000000000000HIR
	JTS100000000000000JOR
	JTS100000000000000LDR
	JTS100000000000000QUR
	JTS100000000000000TSR
	JTS230000000000000HIR
	JTS230000000000000JOR
	JTS230000000000000LDR
	JTS230000000000000QUR
	JTS230000000000000TSR
	JTS300000000000000HIR
	JTS300000000000000JOR
	JTS300000000000000LDR
	JTS300000000000000QUR
	JTS300000000000000TSR
	JTS400000000000000HIR
	JTS400000000000000JOR
	JTS400000000000000LDR
	JTS400000000000000QUR
	JTS400000000000000TSR
	JTS510000000000000HIR
	JTS510000000000000JOR
	JTS510000000000000LDR
	JTS510000000000000QUR
	JTS510000000000000TSR
	JTS510099000000000HIR
	JTS510099000000000JOR
	JTS510099000000000LDR
	JTS510099000000000QUR
	JTS510099000000000TSR
	JTS540099000000000HIR
	JTS540099000000000JOR
	JTS540099000000000LDR
	JTS540099000000000QUR
	JTS540099000000000TSR
	JTS600000000000000HIR
	JTS600000000000000JOR
	JTS6000000

In [149]:
# replace numerical index column with SeriesID
df = df.set_index(['SeriesID'])

# rename seriesIDs with descriptive headers
row_labels={
    # total nonfarm
    "JTS000000000000000HIR": "nonfarm_hires",
    "JTS000000000000000JOR": "nonfarm_openings",
    "JTS000000000000000LDR": "nonfarm_layoffs",
    "JTS000000000000000QUR": "nonfarm_quits",
    "JTS000000000000000TSR": "nonfarm_separations",
    "JTS000000000000000UOR": "nonfarm_unemp_per_opening",
    # nonfarm :: total private
    "JTS100000000000000HIR": "private_hires",
    "JTS100000000000000JOR": "private_openings",
    "JTS100000000000000LDR": "private_layoffs",
    "JTS100000000000000QUR": "private_quits",
    "JTS100000000000000TSR": "private_separations",
    # nonfarm :: private :: construction
    "JTS230000000000000HIR": "construction_hires",
    "JTS230000000000000JOR": "construction_openings",
    "JTS230000000000000LDR": "construction_layoffs",
    "JTS230000000000000QUR": "construction_quits",
    "JTS230000000000000TSR": "construction_separations",
    # nonfarm :: private :: manufacturing
    "JTS300000000000000HIR": "manufacturing_hires",
    "JTS300000000000000JOR": "manufacturing_openings",
    "JTS300000000000000LDR": "manufacturing_layoffs",
    "JTS300000000000000QUR": "manufacturing_quits",
    "JTS300000000000000TSR": "manufacturing_separations",
    # nonfarm :: private :: trade, transportation, utilities
    "JTS400000000000000HIR": "trade_hires",
    "JTS400000000000000JOR": "trade_openings",
    "JTS400000000000000LDR": "trade_layoffs",
    "JTS400000000000000QUR": "trade_quits",
    "JTS400000000000000TSR": "trade_separations",
    # nonfarm :: private :: information
    "JTS510000000000000HIR": "info_hires", 
    "JTS510000000000000JOR": "info_openings", 
    "JTS510000000000000LDR": "info_layoffs",
    "JTS510000000000000QUR": "info_quits",
    "JTS510000000000000TSR": "info_separations",
    # nonfarm :: private :: financial activities
    "JTS510099000000000HIR": "finance_hires",
    "JTS510099000000000JOR": "finance_openings",
    "JTS510099000000000LDR": "finance_layoffs",
    "JTS510099000000000QUR": "finance_quits",
    "JTS510099000000000TSR": "finance_separations",
    # nonfarm :: private :: professional and business services
    "JTS540099000000000HIR": "professional_hires",
    "JTS540099000000000JOR": "professional_openings",
    "JTS540099000000000LDR": "professional_layoffs",
    "JTS540099000000000QUR": "professional_quits",
    "JTS540099000000000TSR": "professional_separations",
    # nonfarm :: private :: education and health
    "JTS600000000000000HIR": "edu_health_hires",
    "JTS600000000000000JOR": "edu_health_openings",
    "JTS600000000000000LDR": "edu_health_layoffs",
    "JTS600000000000000QUR": "edu_health_quits",
    "JTS600000000000000TSR": "edu_health_separations",
    # nonfarm :: total government
    "JTS900000000000000HIR": "gov_hires",
    "JTS900000000000000JOR": "gov_openings",
    "JTS900000000000000LDR": "gov_layoffs",
    "JTS900000000000000QUR": "gov_quits",
    "JTS900000000000000TSR": "gov_separations",
}

df.rename(index=row_labels, inplace=True)
df

,2013-M01,2013-M02,2013-M03,2013-M04,2013-M05,2013-M06,2013-M07,2013-M08,2013-M09,2013-M10,...,2022-M06,2022-M07,2022-M08,2022-M09,2022-M10,2022-M11,2022-M12,2023-M01,2023-M02,2023-M03
SeriesID,,,,,,,,,,,,,,,,,,,,,
nonfarm_hires,3.3,3.4,3.2,3.4,3.4,3.3,3.3,3.5,3.5,3.3,...,4.2,4.1,4.2,4.1,4.0,4.1,4.0,4.1,4.0,4.0
nonfarm_openings,2.8,2.9,2.9,2.9,3.0,3.0,2.8,2.9,2.9,3.0,...,6.7,6.9,6.2,6.6,6.4,6.5,6.8,6.4,6.0,5.8
nonfarm_layoffs,1.2,1.2,1.3,1.3,1.3,1.3,1.3,1.3,1.4,1.2,...,1.0,1.0,1.1,0.9,1.0,1.0,1.0,1.1,1.0,1.2
nonfarm_quits,1.7,1.7,1.6,1.7,1.6,1.6,1.7,1.7,1.7,1.7,...,2.7,2.6,2.8,2.6,2.6,2.7,2.6,2.5,2.6,2.5
nonfarm_separations,3.2,3.2,3.1,3.3,3.3,3.2,3.2,3.3,3.3,3.2,...,3.9,3.8,4.1,3.8,3.8,3.9,3.8,3.8,3.8,3.8
nonfarm_unemp_per_opening,3.2,3.0,2.9,2.9,2.8,2.8,2.9,2.8,2.7,2.6,...,0.5,0.5,0.6,0.5,0.6,0.6,0.5,0.5,0.6,0.6
private_hires,3.7,3.8,3.6,3.8,3.8,3.7,3.7,3.9,3.9,3.7,...,4.6,4.5,4.6,4.4,4.4,4.4,4.4,4.5,4.3,4.3
private_openings,3.0,3.1,3.1,3.0,3.2,3.2,3.0,3.1,3.1,3.2,...,7.1,7.3,6.5,6.9,6.7,6.8,7.1,6.7,6.3,6.0
private_layoffs,1.4,1.4,1.5,1.5,1.5,1.5,1.4,1.5,1.5,1.4,...,1.1,1.1,1.2,1.0,1.1,1.1,1.0,1.2,1.1,1.3


In [166]:
# convert dataframe to csv for Tableau / R analyis
df.to_csv('../files/jolts.csv')

### Notes for Tableau:
- turned time series columns into date variable by pivoting only the month columns
- Series ID | Date | Value
- then can graph different series by graphic series Id vs. Date vs. Value and filtering by series ID
- Problem 1: YYYY-'M'MM is a difficult string format to parse into a date in Tableau, so I have to remove the 'M' from every date column -> regex substitution
- Problem 2: Even after transforming column names in the csv to YYYY-MM, Tableau returned to the previous format YYYY-'M'MM -> split into month and year fields in Tableau, then joined with DATE(Left(Year) +'-'+ Right(Month) + '-1') to set the day for each date value to the 1st; it was getting confused by not having a day to parse 

In [165]:
# convert column names from bytes to strings
df.columns = df.columns.astype(str)

# remove 'M' from column names with regex substitution
df.columns = [re.sub('M', '', column) for column in df.columns]

df

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
SeriesID,,,,,,,,,,,,,,,,,,,,,
nonfarm_hires,3.3,3.4,3.2,3.4,3.4,3.3,3.3,3.5,3.5,3.3,...,4.2,4.1,4.2,4.1,4.0,4.1,4.0,4.1,4.0,4.0
nonfarm_openings,2.8,2.9,2.9,2.9,3.0,3.0,2.8,2.9,2.9,3.0,...,6.7,6.9,6.2,6.6,6.4,6.5,6.8,6.4,6.0,5.8
nonfarm_layoffs,1.2,1.2,1.3,1.3,1.3,1.3,1.3,1.3,1.4,1.2,...,1.0,1.0,1.1,0.9,1.0,1.0,1.0,1.1,1.0,1.2
nonfarm_quits,1.7,1.7,1.6,1.7,1.6,1.6,1.7,1.7,1.7,1.7,...,2.7,2.6,2.8,2.6,2.6,2.7,2.6,2.5,2.6,2.5
nonfarm_separations,3.2,3.2,3.1,3.3,3.3,3.2,3.2,3.3,3.3,3.2,...,3.9,3.8,4.1,3.8,3.8,3.9,3.8,3.8,3.8,3.8
nonfarm_unemp_per_opening,3.2,3.0,2.9,2.9,2.8,2.8,2.9,2.8,2.7,2.6,...,0.5,0.5,0.6,0.5,0.6,0.6,0.5,0.5,0.6,0.6
private_hires,3.7,3.8,3.6,3.8,3.8,3.7,3.7,3.9,3.9,3.7,...,4.6,4.5,4.6,4.4,4.4,4.4,4.4,4.5,4.3,4.3
private_openings,3.0,3.1,3.1,3.0,3.2,3.2,3.0,3.1,3.1,3.2,...,7.1,7.3,6.5,6.9,6.7,6.8,7.1,6.7,6.3,6.0
private_layoffs,1.4,1.4,1.5,1.5,1.5,1.5,1.4,1.5,1.5,1.4,...,1.1,1.1,1.2,1.0,1.1,1.1,1.0,1.2,1.1,1.3


## Step 4: Data Analysis

### 1. Exploratory Data Analysis


#### Tableau Trend Exploration

1. Hires
    1. Spring 2020 (Covid Quarantine)
        - Big Picture: most sectors experienced some volatility between April and July 2020, but most increased their hires in the months after 
        - gov, info, finance, and professional went down sharply 4/1/2020 then bounced back within a few months
        - edu/health and construction went way down (2.4) then way up (5.1) then back to previous levels and steadily increased
        - manufacturing, trade only increased sharply then dropped to previous levels and steadily increased
    2. Hires 10/1/2022 - Present (last 6 months)
        - Big Picture: the professional, construction, and trade/utility sectors are hiring the most, information and construction are the most volatile, finance and government are hiring the least
        - professional > construction > trade > edu/health > manufacturing, information, finance, gov
        - Professional stable around 5.1
        - Construction volatile 5.1-> 4.3 -> 5.1
        - Trade volatile and decreasing 4.3 ^ˇ 4.1
        - Total nonfarm (all) steady at 4.1
        - Edu/health mostly steady at ~3.4
        - Manufacturing mostly steady at 3.0-3.2
        - Information somewhat volatile 3.6 -> 2.7 -> 3.5
        - Finance decreasing 2.4 -> 2.1
        - Government mostly steady 1.7-1.8
2. Layoffs
    1. Layoffs 10/1/2022 - Present (last 6+ months)
        - Big Picture: most industries are fairly volatile in terms of layoffs, with amounts changing by .1-.5 month-to-month in many industries, though a big month is usually followed by a lower month
        - construction > professional > info > trade > manufacturing > edu/health > finance > gov
        - construction having a spike since January (2.0-3.7)
3. Openings
    1. Openings 10/1/2022 - Present (last 6+ months)
        - Big Picture: professional > edu/health >> info* > manufacturing ~ finance > trade > gov > construction
        - edu and health are at the top but slowly coming down from a high in March 21
        - most volatility in info and construction
        - government is consistently low/bottom
        - finance is volatile but holding in the middle of the pack
4. Quits
    1. Quits 10/1/22 - Present
        - Big Picture: trade > professional > edu/health > construction > manufacturing > info > finance > gov
5. Total Separations
    1. Total Separations 10/1/22 - Present
        - Big Picture: professional >~ construction > trade > edu/health ~ manufacturing ~ info > finance > gov
        - don't change our perspective of layoffs and quits much, though it's interesting that professional is far and above 1st with a slow decline since 11/21


Big Picture:
- **hiring**: professional > construction > trade > edu/health > manufacturing, info, finance, gov
- **openings**: professional > edu/health >> info* > manufacturing ~ finance > trade > gov > construction
- **layoffs**: construction > professional > info > trade > manufacturing > edu/health > finance > gov
- **quits**: trade > professional > edu/health > construction > manufacturing > info > finance > gov
- **total separations**: professional >~ construction > trade > edu/health ~ manufacturing ~ info > finance > gov


Industry Take-Aways:
1. Government and Finance are hiring and firing the least people -> low turnover?
    - government also consistently has some of the fewest openings while finance is middle of the pack
    - government and finance have the lowest quits as well
2. Construction and Professional are hiring and firing the most people -> high turnover?
    - construction is usually 2nd to last in openings with recent volatility (not looking for as many people as they are losing?)
    - professional also has the highest openings -> high turnover!
    - professional has the 2nd most quits as well, consistently high
    - construction is mid in quits, volatile
3. Edu/Health are mid-high in hiring and low in layoffs -> understaffed?
    - edu/health has 2nd most openings -> hiring decent numbers of new people, not firing anyone, has lots of openings
    - 3rd in quits, consistently mid-high, fair number of quits every month
4. Info went from 3rd in openings in November to last in December, had a moderate but steady drop in new hires during that time, and was 3rd in layoffs after a 30% uptick in September -> layoffs then hiring freezes
    - volatile openings but generally 3rd, lowest hires in December
    - 2nd/3rd lowest quits -> those who have the jobs and haven't been fired are mostly staying (high months could correlate with big layoffs where people leave during periods of extreme change

Big Thoughts:
- Government and finance are interesting in their stability
- Professional is having consistently high turnover
- Construction is very volatile with turnover and lower openings and only middle in quits
- Edu/Health appear to be understaffed with mid-high hires, low layoffs, lots of openings, and mid-high quits
- Information had recent high layoffs in September and a steady drop in new hires since, still mid-low in hires but layoffs seem to be easing with a drop in March, and openings are on the rise after a sharp drop in December

### 2. Diagnostic Analysis: Last 8 months

I mapped all statistics for the most interesting industries that seem to have a story appearing in their trends (most interesting at the top)

General Relative Trend:
- Openings is 1-4pp higher than hires
- Hires and Total Separations are usually very similar and move together
- Usually more quits than layoffs

1. Information:
- more volatility but trends remain similar (relative statistics)
- consistently much more openings than separations and hires, though had a sharp bounce down in december to lowest since spring 2020
- roughly similar hires and separations, slightly more hires
- generally more quits than layoffs, but has flipped since October (round of layoffs)

2. Edu/Health:
- very steady across the board, not much volatility
- consistently ~2x as many openings as separations and hires (understaffed)
- way more quits than the very small number of layoffs
- slightly more hires than separations (increasing employment)

3. Professional:
- very steady with noticable change since December
- smaller gap between openings and hires
- slightly more hires than separations
- more quits than layoffs
- December-February: drop in openings, hires, total separations, but increase in layoffs

4. Construction:
- about the same number of openings as separations and hires (nearly full employment?)
- quits/layoffs are volatile and swap
- last several months, more separations than hires and drop in openings, big round of layoffs
- recent constriction of the sector

5. Finance:
- trends are low and stable
- 2x openings to hires with recent drop to 1.5x
- roughly equal hires and separations
- more quits than layoffs (very few)
- November: round of layoffs, then drop in openings and hires

6. Government:
- very low and stable
- way more openings than hires & separations (+2x)
- extremely stable and low quits, though still more than very very low layoffs
- not as affected or changed the last 10 months, only a slight increase in turnover (hires & separations) in December

7. Manufacturing:
- usual relative trends
- slight decrease in openings over time

8. Trade
- closer to full employment with small gap between openings and hires/separations
- November: slight increase in quits and openings, now decreasing in all stats

# Future Revisions / Expansions
1. Add education & health specific series as they have been industries of great interest in the labor market lately
2. Look at raw numbers to see how many openings/hires/quits for important months/industries (e.g. Information sector https://www.bls.gov/iag/tgs/iag51.htm)
3. Look at labor participation in these industries -> who could be working in them but is choosing not to or are moving to different industries?
4. Take out 'total separations' and replace with 'other separations' to see nuance -> especially in health where leaving from illness/disability is more likely; however in comparing quits + layoffs = total separations - other separations, it seems to be a small fraction of the other two
5. Looking at seasonally unadjusted data would be interesting
6. Expand the sample to more industries; Mining/Logging, Leisure/Hospitality, Other Services

## Step 5: Communication

### Goals:
    1. Using the Job Openings and Labor Turnover Survey, investigate the labor market as a whole in light of the 'Great Resignation'; which aspects of the GR have continued and which have changed?
    3. Investigage recent layoffs and hiring freezes

### Process:
##### Jupyter Notebook
    1. Data Collection:
        - Found data on Bureau of Labor Statistics website
        - Decided to use API for replicable data collection
        - Imported into pandas dataframe
    2. Data Cleaning:
        - converted series IDs into descriptive headers
##### Tableau
        - pivoted monthly columns to a single date field
        - converted YYYY-'M'MM format into filterable YYYY-MM format
    3. Data Analysis
        - graphed each statistic with all industries to see relative trends between industries
        - graphed each industry with all statistics to see industry-specific trends

### Conclusions:
    1. Great Resignation & Macro Labor Market 
        - increase in quits and job openings rate in Spring 2021
        - continued increase until Spring 2022
        - gradual loosening of the labor market since, with layoffs and hiring freezes in Fall 2022 - Winter 2023
    2. Total Nonfarm in last 5-6 months 
        - Job openings have decreased by 1.6 million (1pp) since December. Layoffs have increased by .2pp since December. Hires and total separations seem stable.
    3. Industry-Specific Findings
        a. Information has had recent high layoffs in September and a steady drop in new hires since, still mid-low in hires but layoffs seem to be easing with a drop in March, and openings are on the rise after a sharp drop in December
        b. Professional is having consistently high turnover
        c. Edu/Health appear to be understaffed with mid-high hires, low layoffs, high of openings, and mid-high quits
        d. Construction is very volatile with turnover and lower openings and only middle in quits
        e. Government and finance are interesting in their stability and low layoffs